## Network Similarity
This is a notebook to compute the node embeddings for each week of the May and June 2022 subreddits for the cross sectional analysis.

In [ ]:
#This is a notebook to compute the node embeddings for each week of the March and April 2022 subreddits for the cross sectional analysis.

In [2]:
pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install networkx==1.11 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.20.0 requires networkx>=2.8, but you have networkx 1.11 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install fastnode2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 49.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
from fastnode2vec import Graph, Node2Vec
import pickle

In [6]:
import ast

In [7]:
def aggressLangProcessing(week_data):
    week_data['toxicityScore'] = week_data['aggLangDict'].progress_apply( lambda x: 
                ast.literal_eval(str(x))['toxicity'])
    week_data['identityAttackScore'] = week_data['aggLangDict'].progress_apply( lambda x: 
                ast.literal_eval(str(x))['identity_attack'])
    week_data['insultScore'] = week_data['aggLangDict'].progress_apply( lambda x: 
                ast.literal_eval(str(x))['insult'])
    week_data['obsceneScore'] = week_data['aggLangDict'].progress_apply( lambda x: 
                ast.literal_eval(str(x))['obscene'])
    week_data['severeToxicityScore'] = week_data['aggLangDict'].progress_apply( lambda x: 
                ast.literal_eval(str(x))['severe_toxicity'])
    week_data['threatScore'] = week_data['aggLangDict'].progress_apply( lambda x: 
                ast.literal_eval(str(x))['threat'])
    return week_data

def cosine_sim(vector1, vector2):
    return min(1., np.dot(vector1, vector2) / (np.linalg.norm(vector1, ord=2) * np.linalg.norm(vector2, ord=2)))

def calculate_author_receiver_counts_with_progress(df):
    groups = df.groupby(['author', 'receiver'])
    counts = {}

    with tqdm(total=len(groups), desc="Calculating author-receiver counts") as pbar:
        for group_key, group_df in groups:
            counts[group_key] = len(group_df)
            pbar.update(1)

    return pd.Series(counts).rename('numerator')

def embeddings_convert_to_numpyarray(embeddings_file):
    num_nodes = len(embeddings_file.key_to_index)
    embedding_dim = embeddings_file.vector_size
    #create a numpy array
    embeddings_array = np.zeros((num_nodes, embedding_dim))
    print(num_nodes)
    print(embedding_dim)
    print(embeddings_array.shape)
    for i, node in enumerate(embeddings_file.key_to_index):
        embeddings_array[i] = embeddings_file[node]
    return embeddings_array

def node_embeddings(week_data, week_no):
    #calculate the edge
    week_data['denominator'] = week_data['receiver'].map(week_data['receiver'].value_counts())
    week_data_1 = calculate_author_receiver_counts_with_progress(week_data)
    week_data_2 = week_data_1.to_frame()
    week_data_2 = week_data_2.reset_index() 
    week_data_2 = week_data_2.rename(columns={"level_0": "author", "level_1": "receiver"})
    week_data_3 = week_data.merge(week_data_2, how='left')
    week_data_3['edge_weights'] = week_data_3['numerator']/week_data_3['denominator']
    print((week_data_3['edge_weights'] <= 1).all())

    #drop duplicates and get a new data set with just unique author-receiver pairs to form the network
    week_data_4 = week_data_3[['author','receiver','edge_weights']]
    graph_df = week_data_4.drop_duplicates(subset = ['author','receiver']).reset_index(drop = True)
    list1 = list(graph_df.itertuples(index=False, name=None))
    graph = Graph(list1, directed=True, weighted=True)
    n2v_q3_week = Node2Vec(graph, dim=128, walk_length=80, window=10, p=1.0, q=3.0, workers=3)
    print('starting the training')
    n2v_q3_week.train(epochs=25)
    model_storage_name_location = 'n2v_models/n2vmodel_week'+ week_no + '.pkl'
    # Open the file in binary write mode
    with open(model_storage_name_location, 'wb') as f:
        pickle.dump(n2v_q3_week.wv, f)
    embeddings_array = embeddings_convert_to_numpyarray(n2v_q3_week.wv)
    print(type(embeddings_array))
    #code to save the embeddings array itself.
    #with open("embeddings.pickle", "wb") as f:
        # Pickle the embeddings using the highest protocol version (optional)
        #pickle.dump(embeddings_array, f, pickle.HIGHEST_PROTOCOL)
    week_data['networkSimilarity'] = week_data.progress_apply(lambda x: cosine_sim(n2v_q3_week.wv[x.author], n2v_q3_week.wv[x.receiver]), axis=1)
    week_data1 = week_data[['id', 'subreddit', 'body', 'author', 'parent_id', 'link_id', 'receiver',
       'receiver_body', 'date', 'toxicityScore',
       'identityAttackScore', 'insultScore', 'obsceneScore',
       'severeToxicityScore', 'threatScore', 'week', 'networkSimilarity']]
    resulting_file_name = 'netSim_processed/mayjune_w'+week_no+'_processed.parquet'
    week_data1.to_parquet(resulting_file_name)

    print('completed!')

In [ ]:
###########modify the following


#node_embeddings('janfeb_w2','2')
#read the processed file with network similarity
#file_name1 = 'janfeb_w3'
#file_path1 = 'JanFebSubreddits2022/weekly_data/' + file_name1 + '.parquet'
#week_data2 = pd.read_parquet(file_path1)
#week_data2.head(2)

#print(len(week_data2))

#read the node2vec model
#file_name1 = 'n2vmodel_week3'
#file_path1 = 'JanFebSubreddits2022/weekly_data/' + file_name1 + '.pkl'
#read_model = pd.read_pickle(file_path1)
#read_model['automoderator']

In [8]:
pip install tqdm


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from tqdm import tqdm
tqdm.pandas()

## Process Week 18.

In [10]:
#readthe data
w18_part1 = pd.read_parquet('weekly_data/MayJune_w18_part1')
w18_part2 = pd.read_parquet('weekly_data/MayJune_w18_part2')
w18 = pd.concat([w18_part1,w18_part2], ignore_index=True, axis=0)
print(len(w18))
w18.head(2)

1008518


,id,subreddit,body,author,created_utc,parent_id,link_id,receiver,receiver_body,receiver_created_utc,date,time,week,aggLangDict
0,i6ukmz0,politics,Fingers crossed. Get your passports arranged,barflyondabeach,1651363290,i6uki4c,ufc7kc,platinumtaters7,I can't wait for the collapse. Sane Americans ...,1651363218,2022-05-01,00:01:30,week18,"{'identity_attack': 0.0001583859120728448, 'in..."
1,i6ukp3m,amitheasshole,^^^^AUTOMOD ***Thanks for posting! This comme...,automoderator,1651363320,ufngd4,ufngd4,disastrous_length_,WIBTA If i told my uncle he cant move in with ...,1651363319,2022-05-01,00:02:00,week18,"{'identity_attack': 0.0003269683802500367, 'in..."


In [11]:
w18 = aggressLangProcessing(w18)
print(len(w18))
w18.head(2)

1008518


100%|██████████| 1008518/1008518 [00:41<00:00, 24456.23it/s]


,id,subreddit,body,author,created_utc,parent_id,link_id,receiver,receiver_body,receiver_created_utc,date,time,week,aggLangDict,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore
0,i6ukmz0,politics,Fingers crossed. Get your passports arranged,barflyondabeach,1651363290,i6uki4c,ufc7kc,platinumtaters7,I can't wait for the collapse. Sane Americans ...,1651363218,2022-05-01,00:01:30,week18,"{'identity_attack': 0.0001583859120728448, 'in...",0.001366,0.000158,0.000198,0.000174,0.000096,0.000125
1,i6ukp3m,amitheasshole,^^^^AUTOMOD ***Thanks for posting! This comme...,automoderator,1651363320,ufngd4,ufngd4,disastrous_length_,WIBTA If i told my uncle he cant move in with ...,1651363319,2022-05-01,00:02:00,week18,"{'identity_attack': 0.0003269683802500367, 'in...",0.026031,0.000327,0.001232,0.002025,0.000153,0.000213


In [12]:
w18['subreddit'].value_counts()

amitheasshole 372661
politics 257921
worldnews 230104
news 118196
science 29636
Name: subreddit, dtype: int64

In [13]:
w18['date'].value_counts()

2022-05-03 222768
2022-05-04 162014
2022-05-02 144216
2022-05-05 140083
2022-05-06 139102
2022-05-07 117116
2022-05-01 83219
Name: date, dtype: int64

In [14]:
node_embeddings(w18,'18')

True
starting the training
248641
128
(248641, 128)
<class 'numpy.ndarray'>
completed!


100%|██████████| 1008518/1008518 [01:07<00:00, 14831.88it/s]


In [15]:
#read the processed file with network similarity
week_data18 = pd.read_parquet('netSim_processed/mayjune_w18_processed.parquet')
week_data18.head(2)

,id,subreddit,body,author,parent_id,link_id,receiver,receiver_body,date,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore,week,networkSimilarity
0,i6ukmz0,politics,Fingers crossed. Get your passports arranged,barflyondabeach,i6uki4c,ufc7kc,platinumtaters7,I can't wait for the collapse. Sane Americans ...,2022-05-01,0.001366,0.000158,0.000198,0.000174,0.000096,0.000125,week18,0.525826
1,i6ukp3m,amitheasshole,^^^^AUTOMOD ***Thanks for posting! This comme...,automoderator,ufngd4,ufngd4,disastrous_length_,WIBTA If i told my uncle he cant move in with ...,2022-05-01,0.026031,0.000327,0.001232,0.002025,0.000153,0.000213,week18,0.352773


## Process Week 19.

In [9]:
#readthe data
w19_part1 = pd.read_parquet('weekly_data/MayJune_w19_part1')
w19_part2 = pd.read_parquet('weekly_data/MayJune_w19_part2')
w19 = pd.concat([w19_part1,w19_part2], ignore_index=True, axis=0)
print(len(w19))
w19.head(2)

950801


,id,subreddit,body,author,created_utc,parent_id,link_id,receiver,receiver_body,receiver_created_utc,date,time,week,aggLangDict
0,i7ql6ye,amitheasshole,Until you're legally an adult it would be disr...,neomiahsmom314,1651968000,ukp51h,ukp51h,azraeltheanarchist,WIBTA To Get A Tattoo Against My Mother's Wishes?,1651966083,2022-05-08,00:00:00,week19,"{'identity_attack': 0.008298913948237896, 'ins..."
1,i7ql74f,amitheasshole,"Oh, I would absolutely follow through with sep...",oldmom64,1651968003,i7q8itd,uknihc,treswhat,NTA. She messed up and now has to earn back mo...,1651961706,2022-05-08,00:00:03,week19,"{'identity_attack': 0.00014794294838793576, 'i..."


In [10]:
w19 = aggressLangProcessing(w19)
print(len(w19))
w19.head(2)

950801


100%|██████████| 950801/950801 [00:38<00:00, 24852.42it/s]


,id,subreddit,body,author,created_utc,parent_id,link_id,receiver,receiver_body,receiver_created_utc,date,time,week,aggLangDict,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore
0,i7ql6ye,amitheasshole,Until you're legally an adult it would be disr...,neomiahsmom314,1651968000,ukp51h,ukp51h,azraeltheanarchist,WIBTA To Get A Tattoo Against My Mother's Wishes?,1651966083,2022-05-08,00:00:00,week19,"{'identity_attack': 0.008298913948237896, 'ins...",0.64323,0.008299,0.120982,0.011706,0.000897,0.003151
1,i7ql74f,amitheasshole,"Oh, I would absolutely follow through with sep...",oldmom64,1651968003,i7q8itd,uknihc,treswhat,NTA. She messed up and now has to earn back mo...,1651961706,2022-05-08,00:00:03,week19,"{'identity_attack': 0.00014794294838793576, 'i...",0.00060,0.000148,0.000185,0.000191,0.000132,0.000133


In [11]:
w19['subreddit'].value_counts()

amitheasshole 399228
worldnews 230103
politics 200954
news 90061
science 30455
Name: subreddit, dtype: int64

In [12]:
w19['date'].value_counts()

2022-05-11 151543
2022-05-13 144938
2022-05-12 137306
2022-05-10 133988
2022-05-09 131684
2022-05-14 129859
2022-05-08 121483
Name: date, dtype: int64

In [13]:
node_embeddings(w19,'19')

True
starting the training
246048
128
(246048, 128)
<class 'numpy.ndarray'>
completed!


100%|██████████| 950801/950801 [00:59<00:00, 15938.18it/s]


In [14]:
#read the processed file with network similarity
week_data19 = pd.read_parquet('netSim_processed/mayjune_w19_processed.parquet')
week_data19.head(2)

,id,subreddit,body,author,parent_id,link_id,receiver,receiver_body,date,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore,week,networkSimilarity
0,i7ql6ye,amitheasshole,Until you're legally an adult it would be disr...,neomiahsmom314,ukp51h,ukp51h,azraeltheanarchist,WIBTA To Get A Tattoo Against My Mother's Wishes?,2022-05-08,0.64323,0.008299,0.120982,0.011706,0.000897,0.003151,week19,0.882737
1,i7ql74f,amitheasshole,"Oh, I would absolutely follow through with sep...",oldmom64,i7q8itd,uknihc,treswhat,NTA. She messed up and now has to earn back mo...,2022-05-08,0.00060,0.000148,0.000185,0.000191,0.000132,0.000133,week19,0.316218


## Process Week 20.

In [10]:
#readthe data
w20_part1 = pd.read_parquet('weekly_data/MayJune_w20_part1')
w20_part2 = pd.read_parquet('weekly_data/MayJune_w20_part2')
w20 = pd.concat([w20_part1,w20_part2], ignore_index=True, axis=0)
print(len(w20))
w20.head(2)

927040


,id,subreddit,body,author,created_utc,parent_id,link_id,receiver,receiver_body,receiver_created_utc,date,time,week,aggLangDict
0,i8mxzz6,worldnews,Stop trying to pretend you know anything.,maplethor,1652572800,i8m9by5,upe4oe,knightofarose,If you think the fullness of intelligence rega...,1652561229,2022-05-15,00:00:00,week20,"{'identity_attack': 0.00031993052107281983, 'i..."
1,i8my0d9,worldnews,Jerusalem was Jewish before humanity invented ...,krneki12,1652572805,i8ks9a6,upfpc7,r3miel7,Can’t wait for you to learn about Israel then,1652536688,2022-05-15,00:00:05,week20,"{'identity_attack': 0.0037340326234698296, 'in..."


In [11]:
w20 = aggressLangProcessing(w20)
print(len(w20))
w20.head(2)

927040


100%|██████████| 927040/927040 [00:37<00:00, 24941.98it/s]


,id,subreddit,body,author,created_utc,parent_id,link_id,receiver,receiver_body,receiver_created_utc,date,time,week,aggLangDict,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore
0,i8mxzz6,worldnews,Stop trying to pretend you know anything.,maplethor,1652572800,i8m9by5,upe4oe,knightofarose,If you think the fullness of intelligence rega...,1652561229,2022-05-15,00:00:00,week20,"{'identity_attack': 0.00031993052107281983, 'i...",0.028180,0.000320,0.000785,0.000638,0.000111,0.000293
1,i8my0d9,worldnews,Jerusalem was Jewish before humanity invented ...,krneki12,1652572805,i8ks9a6,upfpc7,r3miel7,Can’t wait for you to learn about Israel then,1652536688,2022-05-15,00:00:05,week20,"{'identity_attack': 0.0037340326234698296, 'in...",0.008545,0.003734,0.000641,0.000639,0.000392,0.000378


In [12]:
w20['subreddit'].value_counts()

amitheasshole 379824
worldnews 232927
politics 188510
news 106481
science 19298
Name: subreddit, dtype: int64

In [13]:
w20['date'].value_counts()

2022-05-16 141455
2022-05-20 141099
2022-05-19 135491
2022-05-17 134099
2022-05-18 131478
2022-05-15 124434
2022-05-21 118984
Name: date, dtype: int64

In [14]:
node_embeddings(w20,'20')

True
starting the training
245782
128
(245782, 128)
<class 'numpy.ndarray'>
completed!


100%|██████████| 927040/927040 [00:58<00:00, 15857.40it/s]


In [15]:
#read the processed file with network similarity
week_data20 = pd.read_parquet('netSim_processed/mayjune_w20_processed.parquet')
week_data20.head(2)

,id,subreddit,body,author,parent_id,link_id,receiver,receiver_body,date,toxicityScore,identityAttackScore,insultScore,obsceneScore,severeToxicityScore,threatScore,week,networkSimilarity
0,i8mxzz6,worldnews,Stop trying to pretend you know anything.,maplethor,i8m9by5,upe4oe,knightofarose,If you think the fullness of intelligence rega...,2022-05-15,0.028180,0.000320,0.000785,0.000638,0.000111,0.000293,week20,0.976916
1,i8my0d9,worldnews,Jerusalem was Jewish before humanity invented ...,krneki12,i8ks9a6,upfpc7,r3miel7,Can’t wait for you to learn about Israel then,2022-05-15,0.008545,0.003734,0.000641,0.000639,0.000392,0.000378,week20,0.724038
